In [ ]:
%matplotlib inline
from pylab import *
import os
import sys
import glob

# Specific science modules
import healpy as hp
import numpy as np
from astropy.io import fits

# Specific qubic modules
from qubicpack.utilities import Qubic_DataDir
from pysimulators import FitsArray
import pysm
import pysm.units as u

import qubic
from qubic import SpectroImLib as si
from qubic import QubicSkySim as qss
from qubic import NamasterLib as nam
import qubic.fibtools as ft
from qubic.polyacquisition import compute_freq

from qubic import ReadMC as rmc
from qubic import AnalysisMC as amc

rc('figure', figsize=(13, 10))
rc('font', size=15)

In [ ]:
import matplotlib.colors as mcolors
colorlist = list(mcolors.TABLEAU_COLORS.keys())

## Get files

In [ ]:
global_dir = Qubic_DataDir(datafile='instrument.py', datadir=os.environ['QUBIC_DATADIR'])
print(global_dir)

datadir = os.environ['DATA_SPECTROIM']
print(datadir)

In [ ]:
stokes = ['I', 'Q', 'U']
allrecon = [1, 2, 3, 4, 5, 8]

### Full pipeline simu

In [ ]:
# For full pipeline simulations
nfrecon = 3
rep = datadir + f'NERSC_simuEnd2End/Bad_photon_noise2020/simu_NERSC_MC/nfrecon{nfrecon}/'

dictionary = glob.glob(rep + '/*.dict')[0]
print(dictionary)

# Dictionary saved during the simulation
d = qubic.qubicdict.qubicDict()
d.read_from_file(dictionary)

# Coordinates of the zone observed in the sky
center = qubic.equ2gal(d['RA_center'], d['DEC_center'])

# Number of subbands used during the simulation
nfsub = d['nf_sub']
print('nf_sub = {}, nf_recon = {}'.format(nfsub, nfrecon))

# Get fits files names in a list
fits_noise = np.sort(glob.glob(rep + '*_nfsub{}_nfrecon{}_noiselessFalse*.fits'.format(nfsub, nfrecon), 
                              recursive=True))
fits_noiseless = np.sort(glob.glob(rep + '*_nfsub{}_nfrecon{}_noiselessTrue*.fits'.format(nfsub, nfrecon), 
                              recursive=True))

print(fits_noiseless)

# Number of noise realisations
nreals = len(fits_noise)
print('nreals = ', nreals)

# Get seen map (observed pixels)
seenmap_full = rmc.get_seenmap(fits_noiseless[0])
print(seenmap_full.shape)

# Number of pixels and nside
npix = seenmap_full.sum()
print('npix:', npix)

# Patch containing only the noise
patch = amc.get_residuals(fits_noise, fits_noiseless[0], 'noiseless')
print(patch.shape)

### Fast Simulator data

In [ ]:
nfrecon = 2
config = 'FI150'

rep_mapCC = '/home/lmousset/QUBIC/Qubic_work/SpectroImagerie/Simu_spectroim/FastSimulator/maps_FastSim/'

patch = np.load(rep_mapCC + f'noisepatch_NOspatialcorr_nbands{nfrecon}_' + config + '_500reals.npy')
print(patch.shape)
nreals = patch.shape[0]

In [ ]:
# Frequencies of the reconstructed subbands
for nf in allrecon:
    Nfreq_edges, nus_edge, nus, deltas, Delta, Nbbands = compute_freq(int(config[-3:]), nf)
    print(f'nbands = {nf}', nus)

## Coverage and Seenmap

### If simu from FastSimulator

In [ ]:
import pickle
# The coverage is the same for all numbers of bands
DataFastSimCoverage = pickle.load(open(global_dir +
                                       '/doc/FastSimulator/Data/DataFastSimulator_' + config + '_coverage.pkl',
                                       "rb"))
coverage = DataFastSimCoverage['coverage'] # Already normalized
print(coverage.shape)

seenmap_full = coverage > np.max(coverage) * 0.1

coverage_norm = coverage[seenmap_full]
print(coverage_norm.shape)

ang = rmc.pix2ang(256, center, seenmap_full)
print(ang.shape)

# Number of pixels
npix = seenmap_full.sum()
print('npix:', npix)

ang.shape

### If simu from Fullpipeline

In [ ]:
fitsfile = fits.open(fits_noise[0])
coverage = fitsfile['COVERAGE'].data
hp.mollview(coverage)

coverage_patch = coverage[seenmap_full]
print(coverage_patch.shape)

ang = rmc.pix2ang(d['nside'], center, seenmap_full)
coverage_norm = coverage_patch / np.max(coverage_patch)
unseen = coverage_norm < np.max(coverage_norm) * 0.1
seenmap = np.invert(unseen)# Only the patch

plt.figure()
plt.plot(ang, 1 / coverage_norm, '.', label='$1 / coverage$')
plt.plot(ang, 1. / (np.sqrt(coverage_norm)), '.', label='$1 / \sqrt{coverage}$')
plt.xlabel('Angle [deg]')
# plt.ylabel('Coverage normalized')
plt.title('Coverage normalized to the maximum as function of the angle in the patch.')
plt.legend()

## Recombine bands at map level

### Check we get the same using Cp, Cp' or Cp'00

In [ ]:
Cp = amc.get_Cp(patch, verbose=False)
print('Cp shape:', Cp.shape)

# Using Cp directly
avg, sig2 = amc.make_weighted_av(patch[:, ...], Cp, verbose=True)
sig2MC = np.var(avg, axis=0)

In [ ]:
# Cp_prime over bins (normalisation by the first term)
N00, Cp_prime00 = amc.Cp2Cp_prime(Cp, verbose=True)

# Using Cp_prime 00
avg00, sig200 = amc.make_weighted_av(patch[:, ...], Cp_prime00, verbose=True)
sig2MC00 = np.var(avg00, axis=0)


In [ ]:
# Cp_prime over bins (normalisation by the diagonal)
N, Cp_prime = amc.Cp2Cp_prime_viaCorr(Cp, verbose=True)

# Using Cp_prime
avg_prime, sig2_prime = amc.make_weighted_av(patch[:, ...], Cp_prime, verbose=True)
sig2MC_prime = np.var(avg_prime, axis=0)

In [ ]:
x = 1 / coverage_norm[seenmap]

xc, yc, dx, dy, _ = ft.profile(x, sig2[:, 0], 
                               nbins=30, 
                               plot=False,
                               clip=True)


xcMC, ycMC, dxMC, dyMC, _ = ft.profile(x, sig2MC[:, 0], 
                                       nbins=30, 
                                       plot=False,
                                       clip=True)

xc00, yc00, dx00, dy00, _ = ft.profile(x, sig200[:, 0], 
                               nbins=30, 
                               plot=False,
                               clip=True)


xcMC00, ycMC00, dxMC00, dyMC00, _ = ft.profile(x, sig2MC00[:, 0], 
                                       nbins=30, 
                                       plot=False,
                                       clip=True)

xcprime, ycprime, dxprime, dyprime, _ = ft.profile(x, sig2_prime[:, 0], 
                               nbins=30, 
                               plot=False,
                               clip=True)


xcMCprime, ycMCprime, dxMCprime, dyMCprime, _ = ft.profile(x, sig2MC_prime[:, 0], 
                                       nbins=30, 
                                       plot=False,
                                       clip=True)


plt.errorbar(xc, yc, xerr=dx, yerr=dy, 
             color=colorlist[0], label='$\sigma^2$')
plt.errorbar(xcMC, ycMC, xerr=dxMC, yerr=dyMC, 
             color=colorlist[0], fmt='o', label='$\sigma^2 MC$')

plt.errorbar(xc00, yc00, xerr=dx00, yerr=dy00, 
             color=colorlist[1], label='$\sigma^2$ 00')
plt.errorbar(xcMC00, ycMC00, xerr=dxMC00, yerr=dyMC00, 
             color=colorlist[1], fmt='o', label='$\sigma^2 MC$ 00')

plt.errorbar(xcprime, ycprime, xerr=dxprime, yerr=dyprime, 
             color=colorlist[2], label='$\sigma^2$ prime')
plt.errorbar(xcMCprime, ycMCprime, xerr=dxMCprime, yerr=dyMCprime, 
             color=colorlist[2], fmt='o', label='$\sigma^2 MC$ prime')

# plt.ylim(0, 10)

plt.xlabel('$1 / coverage$', fontsize=15)
plt.ylabel('$\sigma^2$', fontsize=15)
plt.legend()
plt.title(f'I - {nbands} bands')

## Recombination using Cp or Cp'

### Fast Simulator data

In [ ]:
# Choose only pixel above a given coverage
coveragecut = 0.3
condition = coverage_norm > coveragecut

In [ ]:
# Recombination (very long, you can use files already saved)
# With Fast Simulator, Cp and Cp' give the same so you can use Cp.

mylist_sig2, mylist_sig2MC = [], []
mylist_Cp, mylist_Cp_prime = [], []

for i in range(6):
    nfrecon = allrecon[i]
    print('\n nfrecon:', nfrecon)
    
    patch = np.load(rep_mapCC + f'noisepatch_NOspatialcorr_nbands{nfrecon}_' + config + '_500reals.npy')
    
    patch = patch[:, :, condition, :]
    print('patch shape:', patch.shape)

    nreals, _, npix, _ = patch.shape
    print('npix:', npix)
    print('# realisations:', nreals)
    
    Cp = amc.get_Cp(patch, verbose=False)
    print('Cp shape:', Cp.shape)
    mylist_Cp.append(Cp)
    print(Cp[:, :, 0, 0])
    
#     N, Cp_prime = amc.Cp2Cp_prime_viaCorr(Cp)
#     mylist_Cp_prime.append(Cp_prime)
#     print('Cp_prime shape:', Cp_prime.shape)
    
    avg, sig2 = amc.make_weighted_av(patch[:, ...], Cp, verbose=True)
    sig2MC = np.var(avg, axis=0)
    mylist_sig2.append(sig2)
    mylist_sig2MC.append(sig2MC)    

In [ ]:
# We save sig2 and sig2MC because especially sig2MC is long to compute.
np.save(rep_mapCC + f'sig2_' + config + f'_coveragecut{coveragecut}_Cp.npy', mylist_sig2)
np.save(rep_mapCC + f'sig2MC_' + config + f'_coveragecut{coveragecut}_Cp.npy', mylist_sig2MC)

### Full pipeline data

In [ ]:
coveragecut = 0.3

In [ ]:
# Recombination
# Cp and Cp' don't give exactly the same...

mylist_sig2, mylist_sig2MC = [], []
mylist_Cp, mylist_Cp_prime = [], []
myang, mycoverage_norm = [], []

for i in range(6):
    nfrecon = allrecon[i]
    print('\n nfrecon:', nfrecon)
    
    rep = datadir + f'NERSC_simuEnd2End/Bad_photon_noise2020/simu_NERSC_MC/nfrecon{nfrecon}/'
    
    dictionary = glob.glob(rep + '/*.dict')[0]
    print(dictionary)

    # Dictionary saved during the simulation
    d = qubic.qubicdict.qubicDict()
    d.read_from_file(dictionary)

    # Number of subbands used during the simulation
    nfsub = d['nf_sub']
    print('nf_sub = {}, nf_recon = {}'.format(nfsub, nfrecon))

    # Get fits files names in a list
    fits_noise = np.sort(glob.glob(rep + '*_nfsub{}_nfrecon{}_noiselessFalse*.fits'.format(nfsub, nfrecon), 
                                  recursive=True))
    fits_noiseless = np.sort(glob.glob(rep + '*_nfsub{}_nfrecon{}_noiselessTrue*.fits'.format(nfsub, nfrecon), 
                                  recursive=True))

    # Coverage and seenmap
    seenmap_full = rmc.get_seenmap(fits_noiseless[0])

    fitsfile = fits.open(fits_noise[0])
    coverage = fitsfile['COVERAGE'].data

    coverage_patch = coverage[seenmap_full]

    ang = rmc.pix2ang(d['nside'], center, seenmap_full)
    myang.append(ang)
    coverage_norm = coverage_patch / np.max(coverage_patch)
    mycoverage_norm.append(coverage_norm)
     
#     # Choose only pixel above a given coverage
#     condition = coverage_norm > coveragecut

#     # Patch containing only the noise
#     patch = amc.get_residuals(fits_noise, fits_noiseless[0], 'noiseless')
#     print(patch.shape)
    
#     patch = patch[:, :, condition, :]
#     print('patch shape:', patch.shape)

#     nreals, _, npix, _ = patch.shape
#     print('npix:', npix)
#     print('# realisations:', nreals)
    
#     # Cp, Cp' and map recombination
#     Cp = amc.get_Cp(patch, verbose=False)
#     print('Cp shape:', Cp.shape)
#     mylist_Cp.append(Cp)
#     print(Cp[:, :, 0, 0])
    
#     N, Cp_prime = amc.Cp2Cp_prime_viaCorr(Cp)
#     mylist_Cp_prime.append(Cp_prime)
#     print('Cp_prime shape:', Cp_prime.shape)
    
#     avg, sig2 = amc.make_weighted_av(patch[:, ...], Cp, verbose=True)
#     sig2MC = np.var(avg, axis=0)
#     mylist_sig2.append(sig2)
#     mylist_sig2MC.append(sig2MC)  

In [ ]:
# We save sig2 and sig2MC because especially sig2MC is long to compute.
myrep = datadir + 'NERSC_simuEnd2End/Bad_photon_noise2020/simu_NERSC_MC/'
# np.save(myrep + f'sig2_fullpipeline_coveragecut{coveragecut}_Cp.npy', mylist_sig2)
# np.save(myrep + f'sig2MC_fullpipeline_coveragecut{coveragecut}_Cp.npy', mylist_sig2MC)

### Profile over pixels, make bins

In [ ]:
# Load files already saved
# mylist_sig2 = np.load(myrep + 'sig2_fullpipeline_coveragecut0.3_Cp_prime.npy', allow_pickle=True)
# mylist_sig2MC = np.load(myrep + 'sig2MC_fullpipeline_coveragecut0.3_Cp_prime.npy', allow_pickle=True)

mylist_sig2 = np.load(rep_mapCC + f'sig2_FI150_coveragecut0.3_Cp.npy')
mylist_sig2MC = np.load(rep_mapCC + f'sig2MC_FI150_coveragecut0.3_Cp.npy')

In [ ]:
# Profile over pixels
nbins = 15
x = np.zeros((6, nbins, 3))
y = np.zeros_like(x)
dx = np.zeros_like(x)
dy = np.zeros_like(x)
xMC = np.zeros_like(x)
yMC = np.zeros_like(x)
dxMC = np.zeros_like(x)
dyMC = np.zeros_like(x)

a = ang[coverage_norm > coveragecut]#1 / np.sqrt(coverage_norm[seenmap][condition])

for i in range(6):
    sig = np.sqrt(mylist_sig2[i]) 
    sigMC = np.sqrt(mylist_sig2MC[i])
    print('sigma',sig.shape)
    
#     a = myang[i][mycoverage_norm[i]>coveragecut]
#     a = 1 / np.sqrt(mycoverage_norm[i][mycoverage_norm[i]>coveragecut])
#     print(mycoverage_norm[i].shape)
#     print(mycoverage_norm[i][mycoverage_norm[i]>coveragecut].shape)
    for istk in range(3):        
        x[i, :, istk], y[i, :, istk], dx[i, :, istk], dy[i, :, istk], _ = ft.profile(a, 
                                                                                     sig[:, istk], 
                                                                                     nbins=nbins, 
                                                                                     dispersion=True,
                                                                                     plot=False,
                                                                                     clip=True)


        xMC[i, :, istk], yMC[i, :, istk], dxMC[i, :, istk], dyMC[i, :, istk], _ = ft.profile(a, 
                                                                                             sigMC[:, istk],
                                                                                             nbins=nbins, 
                                                                                             dispersion=True,
                                                                                             plot=False,
                                                                                             clip=True)


In [ ]:
xydxdy = [x, y, dx, dy]
xydxdyMC = [xMC, yMC, dxMC, dyMC]

# np.save(myrep + f'xydxdy_angle_fullpipeline_coveragecut{coveragecut}_Cp_prime.npy', xydxdy)
# np.save(myrep + f'xydxdyMC_angle_fullpipeline_coveragecut{coveragecut}_Cp_prime.npy', xydxdyMC)

np.save(rep_mapCC + f'xydxdy_angle_FastSim_150_coveragecut{coveragecut}_Cp.npy', xydxdy)
np.save(rep_mapCC + f'xydxdyMC_angle_FastSim_150_coveragecut{coveragecut}_Cp.npy', xydxdyMC)

In [ ]:
x, y, dx, dy = np.load(myrep + f'xydxdy_angle_fullpipeline_coveragecut{coveragecut}_Cp_prime.npy')
xMC, yMC, dxMC, dyMC = np.load(myrep + f'xydxdyMC_angle_fullpipeline_coveragecut{coveragecut}_Cp_prime.npy')

In [ ]:
import seaborn as sns

palet = sns.cubehelix_palette(6, start=1, hue=1, light=0.75)
sns.palplot(palet)


In [ ]:
plt.figure(figsize=(20, 10))
for istk in range(3):
    plt.subplot(1, 3, istk+1)
    for i in range(6):
        c = palet[i]
        plt.errorbar(x[i, :, istk], y[i, :, istk], xerr=dx[i, :, istk], yerr=dy[i, :, istk], 
                     color=c, label='theo - {} bands'.format(allrecon[i]))
        plt.errorbar(xMC[i, :, istk], yMC[i, :, istk], xerr=dxMC[i, :, istk], yerr=dyMC[i, :, istk], 
                     fmt='o', color=c, label='MC - {} bands'.format(allrecon[i]))

#     plt.ylim(0.9, 2.2)
    plt.ylabel('$\sigma$', fontsize=15)
    plt.xlabel(r'$\theta$ [deg.]', fontsize=15)
    plt.legend(loc='best', fontsize=10)
    plt.title('{} component'.format(stokes[istk]))

In [ ]:
# Figure spectroim paper: only Q 
istk = 1
plt.figure(figsize=(10, 8))
for i in [0, 2]:
    c = palet[i+1]
    if i==0:
        bandword = 'sub-band'
    else:
        bandword = 'sub-bands'
    plt.errorbar(x[i, :, istk], 
                 y[i, :, istk], 
                 xerr=dx[i, :, istk], 
                 yerr=dy[i, :, istk], 
                 color=c, 
                 label='$\sigma$ - {} {}'.format(allrecon[i], bandword))
    plt.errorbar(xMC[i, :, istk], 
                 yMC[i, :, istk], 
                 xerr=dxMC[i, :, istk], 
                 yerr=dyMC[i, :, istk], 
                 fmt='o', 
                 color=c, 
                 label='$\sigma_{MC}$'+' - {} {}'.format(allrecon[i], bandword))

#     plt.ylim(0.9, 2.2)
plt.ylabel(r'$\sigma$', fontsize=18)
plt.xlabel(r'$\theta$ [deg.]', fontsize=18)
# plt.xlabel(r'$1 \left/ \sqrt{\mathrm{Cov} \left/ \mathrm{Cov_{MAX}} \right. } \right.$', fontsize=18)
plt.legend(loc='best', fontsize=16)
# plt.title('Stokes parameter {} - {} GHz'.format(stokes[istk], config[-3:]))
plt.title('Stokes parameter {} - 150 GHz'.format(stokes[istk]))
plt.grid()

# plt.savefig('/home/lmousset/QUBIC/Qubic_work/SpectroImagerie/paper_plot/BandRecombi_Q_3bands_' 
#             + config +'_FastSim.pdf')
# plt.savefig('/home/lmousset/QUBIC/Qubic_work/SpectroImagerie/paper_plot/BandRecombi_Q_3bands_fullpipeline.pdf')

In [ ]:
### Figure spectroim paper: Evolution of the first pixel bin
istk = 2
mybin = 4
yerr = 1 / y[0, mybin, istk]**2 * np.sqrt(dy[i, mybin, istk]**2 + dy[0, mybin, istk]**2 * (y[:, mybin, istk]/y[0, mybin, istk])**2)
plt.figure(figsize=(10, 8))
plt.errorbar(allrecon, 
         y[:, mybin, istk] / y[0, mybin, istk],
         yerr=yerr,
        fmt='ro')
# plt.plot(allrecon, yMC[:, 0, istk]/yMC[0, 0, istk], 'o', label=stokes[istk])
# plt.plot(allrecon, np.sqrt(allrecon), 'k', label='$\sqrt{N}$')
plt.xlabel('Number of sub-bands', fontsize=18)
plt.ylabel(r'$\sigma \left/ \sigma_{1\mathrm{band}} \right.$ ', fontsize=18)
plt.title('Stokes parameter {} - {} GHz - Bin {}'.format(stokes[istk], config[-3:], mybin))
# plt.title('Stokes parameter {} - 150 GHz'.format(stokes[istk]))
plt.grid()
plt.savefig(f'/home/lmousset/QUBIC/Qubic_work/SpectroImagerie/paper_plot/BandRecombi_{stokes[istk]}_WithBandNumber_' 
            + config + f'_FastSim_bin{mybin}.pdf')
# plt.savefig(f'/home/lmousset/QUBIC/Qubic_work/SpectroImagerie/paper_plot/BandRecombi_{stokes[istk]}_WithBandNumber_fullpipeline_bin{mybin}.pdf')

### Compare FastSim 150, 220 and Fullpipeline

In [ ]:
x, y, dx, dy = np.load(myrep + f'xydxdy_angle_fullpipeline_coveragecut{coveragecut}_Cp_prime.npy')
xFS150, yFS150, dxFS150, dyFS150 = np.load(rep_mapCC + f'xydxdy_angle_FastSim_150_coveragecut{coveragecut}_Cp.npy')
xFS220, yFS220, dxFS220, dyFS220 = np.load(rep_mapCC + f'xydxdy_angle_FastSim_220_coveragecut{coveragecut}_Cp.npy')

In [ ]:
def error_XoverY(x, y, dx, dy):
    error = 1 / y * np.sqrt(dx**2 + (x/y)**2 * dy**2)
    return error

In [ ]:
istk = 2
mybin = 1
yerr = error_XoverY( y[:, mybin, istk], y[0, mybin, istk], dy[:, mybin, istk], dy[0, mybin, istk])
yerrFS150 = error_XoverY( yFS150[:, mybin, istk], yFS150[0, mybin, istk], dyFS150[:, mybin, istk], dyFS150[0, mybin, istk])
yerrFS220 = error_XoverY( yFS220[:, mybin, istk], yFS220[0, mybin, istk], dyFS220[:, mybin, istk], dyFS220[0, mybin, istk])

plt.figure(figsize=(10, 8))
plt.errorbar(allrecon, 
         y[:, mybin, istk] / y[0, mybin, istk],
         yerr=yerr,
        fmt='ro',
         label='End-to-End 150GHz')
plt.errorbar(allrecon, 
         yFS150[:, mybin, istk] / yFS150[0, mybin, istk],
         yerr=yerrFS150,
        fmt='bo',
            label='Fast Simulator 150GHz')
plt.errorbar(allrecon, 
         yFS220[:, mybin, istk] / yFS220[0, mybin, istk],
         yerr=yerrFS220,
        fmt='go',
            label='Fast Simulator 220GHz')
# plt.plot(allrecon, yMC[:, 0, istk]/yMC[0, 0, istk], 'o', label=stokes[istk])
# plt.plot(allrecon, np.sqrt(allrecon), 'k', label='$\sqrt{N}$')
plt.xlabel('Number of sub-bands', fontsize=18)
plt.ylabel(r'$\sigma \left/ \sigma_{1\mathrm{band}} \right.$ ', fontsize=18)
plt.title('Stokes parameter {} - Bin {}'.format(stokes[istk], mybin))
# plt.title('Stokes parameter {} - 150 GHz'.format(stokes[istk]))
plt.grid()
plt.legend()

### Average the bins

In [ ]:
x[:, :5, :].shape

In [ ]:
# Average the 5 first bins
x_avg = np.mean(x[:, :5, :], axis=1)
y_avg = np.mean(y[:, :5, :], axis=1)

xFS150_avg = np.mean(xFS150[:, :5, :], axis=1)
yFS150_avg = np.mean(yFS150[:, :5, :], axis=1)

xFS220_avg = np.mean(xFS220[:, :5, :], axis=1)
yFS220_avg = np.mean(yFS220[:, :5, :], axis=1)

# Average the errors
dx_avg = np.sqrt(np.sum(dx[:, :5, :]**2, axis=1))
dy_avg = np.sqrt(np.sum(dy[:, :5, :]**2, axis=1))

dxFS150_avg = np.sqrt(np.sum(dxFS150[:, :5, :]**2, axis=1))
dyFS150_avg = np.sqrt(np.sum(dyFS150[:, :5, :]**2, axis=1))

dxFS220_avg = np.sqrt(np.sum(dxFS220[:, :5, :]**2, axis=1))
dyFS220_avg = np.sqrt(np.sum(dyFS220[:, :5, :]**2, axis=1))


In [ ]:
istk = 1

yerr_avg = error_XoverY(y_avg[:, istk], y_avg[0, istk], dy_avg[:, istk], dy_avg[0, istk])
yerrFS150_avg = error_XoverY(yFS150_avg[:, istk], yFS150_avg[0, istk], dyFS150_avg[:, istk], dyFS150_avg[0, istk])
yerrFS220_avg = error_XoverY(yFS220_avg[:, istk], yFS220_avg[0, istk], dyFS220_avg[:, istk], dyFS220_avg[0, istk])

plt.figure(figsize=(10, 8))
plt.errorbar(allrecon, 
         y_avg[:, istk]/y_avg[0, istk],
         yerr=yerr_avg,
        fmt='ro',
         label='End-to-End 150GHz')
plt.errorbar(allrecon, 
         yFS150_avg[:, istk]/yFS150_avg[0, istk],
         yerr=yerrFS150_avg,
        fmt='bo',
            label='Fast Simulator 150GHz')
plt.errorbar(allrecon, 
         yFS220_avg[:, istk]/yFS220_avg[0, istk],
         yerr=yerrFS220_avg,
        fmt='go',
            label='Fast Simulator 220GHz')
# plt.plot(allrecon, yMC[:, 0, istk]/yMC[0, 0, istk], 'o', label=stokes[istk])
# plt.plot(allrecon, np.sqrt(allrecon), 'k', label='$\sqrt{N}$')
plt.xlabel('Number of sub-bands', fontsize=18)
plt.ylabel(r'$\sigma \left/ \sigma_{1\mathrm{band}} \right.$ ', fontsize=18)
plt.title('Stokes parameter {} - Mean over the first 5 bins'.format(stokes[istk]))
# plt.title('Stokes parameter {} - 150 GHz'.format(stokes[istk]))
plt.grid()
plt.legend()
plt.axhline(y=1, ls='--', c='k')

## Recombination with JC matrices

We have to remultiply the matrices as they are normalized (term00=1 for I and 2 for Q, U) whatever the number of bands.
Not sure this part of the code works well...

In [ ]:
npix = 25000
sig2 = np.zeros((6, 3))
avg = np.zeros((npix, 6, 3))

for i in range(6):
    nfrecon = allrecon[i]
    print('\n nfrecon:', nfrecon)
    
#     coverage_norm = coverage_normlist[i] 
    patch = np.load(rep_mapCC + f'noisepatch_NOspatialcorr_nbands{nfrecon}_' + config + '_500reals.npy')[0]
    
#     rep = '/home/lmousset/QUBIC/Qubic_work/SpectroImagerie/Simu_spectroim/simu_nersc_dust_fullpipeline/nfrecon{}/'.format(nfrecon)
#     dictionary = glob.glob(rep + '/*.dict')[0]
#     d = qubic.qubicdict.qubicDict()
#     d.read_from_file(dictionary)

#     # Get fits files names in a list
#     fits_noise = np.sort(glob.glob(rep + '*_nfsub{}_nfrecon{}_noiselessFalse*.fits'.format(d['nf_sub'], nfrecon), 
#                                   recursive=True))
#     fits_noiseless = np.sort(glob.glob(rep + '*_nfsub{}_nfrecon{}_noiselessTrue*.fits'.format(d['nf_sub'], nfrecon), 
#                                   recursive=True))

#     # Get seen map (observed pixels)
#     seenmap = rmc.get_seenmap(fits_noiseless[0])
#     patch = amc.get_residuals(fits_noise, fits_noiseless[0], 'noiseless')[0]
    
#     fitsfile = fits.open(fits_noise[0])
#     coverage = fitsfile['COVERAGE'].data
#     coverage_patch = coverage[seenmap]
#     coverage_norm = coverage_patch / np.max(coverage_patch)
    
    
    covI, covQ, covU, all_fitcov, all_norm_noise, flatten_patch = qss.get_cov_nunu(patch, 
                                                         coverage_norm[seenmap], 
                                                         nbins=20, 
                                                         QUsep=True, 
                                                         return_flat_maps=True)
    print(len(all_fitcov), np.mean(all_fitcov[0], axis=1))
    print(len(all_norm_noise), all_norm_noise)
    factor_back = all_norm_noise[0]**2
    covI *= factor_back
    covQ *= factor_back
    covU *= factor_back
#     covI = np.diag(covI) * np.identity(nfrecon)
#     covQ = np.diag(covQ) * np.identity(nfrecon)
#     covU = np.diag(covU) * np.identity(nfrecon)
    print('covI:', covI)
    covJC = np.asarray([covI, covQ, covU])
    
    for pix in range(npix):
        for istk in range(3):
            avg[pix, i, istk], sig2[i, istk] = amc.get_weighted_correlation_average(patch[:, pix, istk], 
                                                                                     covJC[istk, :, :])
    # Plot
    labs = []
    for nf in range(nfrecon):
        labs.append('f{}'.format(nf))

    plt.figure(figsize=(15, 5))
    for s in range(3):
        plt.subplot(1, 3, s+1)
        vmax = np.max(np.abs(covJC[s, :, :]))
        plt.imshow(covJC[s, :, :], vmin=-vmax, vmax=vmax, cmap='bwr')
        plt.title(stokes[s] + ' - Cov JC')
        plt.colorbar(orientation='horizontal')
        plt.xticks(np.arange(len(labs)), labs)
        plt.yticks(np.arange(len(labs)), labs)

sig2MC = np.var(avg, axis=0)

    

In [ ]:
for i in range(3):
    plt.plot(allrecon, sig2[:, i]/sig2[0, i], color=colorlist[i], label=stokes[i] + ' th')
#     plt.plot(allrecon, sig2MC[:, i]/sig2MC[0, i], 'o', color=colorlist[i], label=stokes[i] + ' MC')

plt.legend()
plt.xlabel('# bands')
plt.ylabel('$\sigma^2/ \sigma^2_{1band}$')

## Small statistical test

In [ ]:
nbands = 2
nreals = 1000

mean = [0, 0]

# Covariance matrix
mycov = np.identity(nbands) #* nbands

mycov += [[0, -0.7],[-0.7, 0.]]
# mycov += [[0., -0.5, 0.],[-0.5, 0., -0.5],[0., -0.5, 0.]]
print(mycov)

x = np.random.multivariate_normal(mean, mycov, size=nreals)
print(x.shape)

avg = np.zeros(nreals)
for r in range(nreals):
    avg[r], sig2 = amc.get_weighted_correlation_average(x[r, :], mycov)

sig2MC = np.var(avg)
print(sig2, sig2MC)